In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [3]:
df= pd.read_csv("C:/Users/DELL/OneDrive/Desktop/anu course/datasets/intrusion1.csv")

In [5]:
df.describe()

,network_packet_size,login_attempts,session_duration,ip_reputation_score,failed_logins,unusual_time_access,attack_detected
count,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000
mean,500.430639,4.032086,792.745312,0.331338,1.517773,0.149942,0.447101
std,198.379364,1.963012,786.560144,0.177175,1.033988,0.357034,0.497220
min,64.000000,1.000000,0.500000,0.002497,0.000000,0.000000,0.000000
25%,365.000000,3.000000,231.953006,0.191946,1.000000,0.000000,0.000000
50%,499.000000,4.000000,556.277457,0.314778,1.000000,0.000000,0.000000
75%,635.000000,5.000000,1105.380602,0.453388,2.000000,0.000000,1.000000
max,1285.000000,13.000000,7190.392213,0.924299,5.000000,1.000000,1.000000


In [7]:
df.isnull().sum()

session_id                0
network_packet_size       0
protocol_type             0
login_attempts            0
session_duration          0
encryption_used        1966
ip_reputation_score       0
failed_logins             0
browser_type              0
unusual_time_access       0
attack_detected           0
dtype: int64

In [9]:
df.columns.tolist()

['session_id',
 'network_packet_size',
 'protocol_type',
 'login_attempts',
 'session_duration',
 'encryption_used',
 'ip_reputation_score',
 'failed_logins',
 'browser_type',
 'unusual_time_access',
 'attack_detected']

In [11]:
df.dtypes

session_id              object
network_packet_size      int64
protocol_type           object
login_attempts           int64
session_duration       float64
encryption_used         object
ip_reputation_score    float64
failed_logins            int64
browser_type            object
unusual_time_access      int64
attack_detected          int64
dtype: object

In [13]:
df['encryption_used'] = df['encryption_used'].fillna(df['encryption_used'].mode()[0] )

In [15]:
df.dtypes


session_id              object
network_packet_size      int64
protocol_type           object
login_attempts           int64
session_duration       float64
encryption_used         object
ip_reputation_score    float64
failed_logins            int64
browser_type            object
unusual_time_access      int64
attack_detected          int64
dtype: object

In [17]:
df.isnull().sum()

session_id             0
network_packet_size    0
protocol_type          0
login_attempts         0
session_duration       0
encryption_used        0
ip_reputation_score    0
failed_logins          0
browser_type           0
unusual_time_access    0
attack_detected        0
dtype: int64

In [19]:
df.head(10)

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
0,SID_00001,599,TCP,4,492.983263,DES,0.606818,1,Edge,0,1
1,SID_00002,472,TCP,3,1557.996461,DES,0.301569,0,Firefox,0,0
2,SID_00003,629,TCP,3,75.044262,DES,0.739164,2,Chrome,0,1
3,SID_00004,804,UDP,4,601.248835,DES,0.123267,0,Unknown,0,1
4,SID_00005,453,TCP,5,532.540888,AES,0.054874,1,Firefox,0,0
5,SID_00006,453,UDP,5,380.471550,AES,0.422486,2,Chrome,1,0
6,SID_00007,815,ICMP,4,728.107165,AES,0.413772,1,Chrome,0,1
7,SID_00008,653,TCP,3,12.599906,DES,0.097719,3,Chrome,1,1
8,SID_00009,406,TCP,2,542.558895,AES,0.294580,0,Chrome,1,0
9,SID_00010,608,UDP,6,531.944107,AES,0.424117,1,Chrome,0,0


In [21]:
X = df.drop('attack_detected', axis=1)
y = df['attack_detected']

In [23]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)


In [25]:
#for numerical features
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Numeric columns :", num_features)

Numeric columns : ['network_packet_size', 'login_attempts', 'session_duration', 'ip_reputation_score', 'failed_logins', 'unusual_time_access']


In [27]:
#for categorical features
cat_features = X.select_dtypes(include='object').columns.tolist()
print("Categorical columns :", cat_features)

Categorical columns : ['session_id', 'protocol_type', 'encryption_used', 'browser_type']


In [29]:
#Pipelining for numerical values
numerical_cols = Pipeline(
    steps=[
        ("Simple impute", SimpleImputer(strategy='mean')),
        ("scaling", StandardScaler())
    ]
)

In [31]:
#Pipelining for categorical values
categorical_cols = Pipeline(
    steps=[
        ("constant imputation", SimpleImputer(strategy='most_frequent')),
        ("ohe", OneHotEncoder(handle_unknown='ignore'))
    ]
)

In [33]:
#combining Pipelines
preprocessing = ColumnTransformer(
    transformers=[
        ("categories", categorical_cols, cat_features),
        ("numerical", numerical_cols, num_features)
    ]
)

In [35]:
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline(
    steps = [("processor",preprocessing),("estimator",RandomForestClassifier())]
)



In [37]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('processor',
                 ColumnTransformer(transformers=[('categories',
                                                  Pipeline(steps=[('constant '
                                                                   'imputation',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['session_id',
                                                   'protocol_type',
                                                   'encryption_used',
                                                   'browser_type']),
                                                 ('numerical',
                                                  Pipeline(steps=[('Simple '
                                                                   'impute',
                                                                   SimpleImputer()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['network_packet_size',
                                                   'login_attempts',
                                                   'session_duration',
                                                   'ip_reputation_score',
                                                   'failed_logins',
                                                   'unusual_time_access'])])),
                ('estimator', RandomForestClassifier())])

In [39]:
pipe.predict(X_test)

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score
y_pred = pipe.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print("Acurracy Score",round(acc * 100,2),"%")

Acurracy Score 89.2 %
